# CiS - Projekt

Micha von der Osten-Sacken<br>
Timo Wacke

## <b>Numerische Berechnung von Planetenbahnen<b>

## Solarsystem

Um ein Solarsystem nach dem Vorbild des Unseren zu simulieren haben wir in der Datei Solar.py die Massen, 
Distanzen zur Sonne und Bahngeschwindigkeiten von den Planeten und unserem Mond definiert.

In [ ]:
#verify that Solar.py and progressBar.py are in the same directory as this jupyter Notebook
import Solar as sol 


## Verfahren
Die Bewegungsgleichung für 3 oder mehr Körper lässt sich im Allgemeinen nicht analytisch lösen.<br>
Numerisch gibt es verschieden Anstäze:

<b>Trivial Euler Verfahren:</b>

Zu einer Startposition $\vec{r}(0)$ mit gegebenem $\vec{v}(0)$ und einem gewählten Zeitschritt $dt$, berechnet man iterativ die Koordinaten für die folgenden Koordinaten nach der Vorschrift:
$$\vec{r}(t+dt)=\vec{r}(t) + dt \cdot \vec{v}(t)$$
wobei sich jeweils das neue $\vec{v}(t)$ für einen nächsten Schritt aus der Kraft $\vec{F}(r)$ welche bei \vec{r}(t) auf den Körper wirkt berechnet.
$$\vec{a}=\vec{F}/m \\ \vec{v}(t+dt)=\vec{v}(t)+\vec{a}(t+dt) \cdot dt$$
Um die weiteren Zeitschritte zu berechnen, ist das erfahren zu Widerholen bzw. $t=t+dt$ zu setzen.<br>
Dieses Verfahren ist aber sehr instabil, denn selbst bei einer konstanten Beschleunigung ist die Geschwindigkeit mit der bei einem Zeitschritt gerechnet wird immer nur für den Anfangszeitpunkt des Zeitschrittes exakt und danach immer zu wenig in die beschleunigte Richtung. Für eine konstante Kraft kommt so in jedem Zeitschritt der gleiche Fehler hinzu und der summiert sich auf und wird immer größer. Beim Beispiel der Planetenbewegung ist die Kraft zwar nicht konstant, aber immer nach innen gerichtet, sodass der Planet bei dem Verfahren immer nicht weit genug nach innen beschleunigt wird, und immer weiter nach außen wegdriftet.
Berechnet man zu erst $\vec{v}(t + dt)$ und mit dieser Geschwindigkeit dann $\vec{r}(t + dt)$ so wird die Beschleunigung zu weit nach innen gerichtet sein, und der Planet immer weiter nach innen driften.<br>
<br>
Durch möglichst kleine Zeitschritte lässt sich der Fehler und damit der Drift zwar minimieren aber nicht beseitigen.
Vergleichbar mit der Unter und Obersumme beim Integrieren.

<b>Besser Verlet Verfahren:</b>

Vor dem ersten Schritt berechnet man die Kraft $\vec{F}(r)$ und die daraus resultierende Beschleunigung $\vec{a}=\vec{F}/m$ die auf den Körper an den entsprechenden Startkoordinaten wirkt.<br>
Dann passt man die Geschwindigkeit nur um die Beschleunigung für einen halben Zeitschritt an: $$\vec{v}(t + ½ dt) = \vec{v}(t) + ½ dt \cdot \vec{a}(t).$$
Wenn wir von nun an $$\vec{r}(t + dt)$$ berechnen machen wir das auf der Grundlage von \vec{v}(t + ½ dt )<br>
Und wenn wir \vec{v}(t + dt + ½ dt ) berechnen machen wir das auf der Grundlage von \vec{F}(t + dt).<br>
Dann werden in jedem Zeitschritt die Koordinaten $\vec{r}(t+dt)=\vec{r}(t)+\vec{v}(t+½ \cdot dt) \cdot dt$ anhand der Geschwindigkeit zum Zeitpunkt in der Mitte des Zeitschrittes berechnet. Und die Geschwindigkeit $\vec{v}(t+3/2 \cdot dt)=\vec{v}(t+½ \cdot dt)+\vec{a}(t+dt) \cdot dt$ wird anhand der Beschleunigung aus der Kraft zu den Koordinaten von ganzzahligen Zeitschritten berechnet, wobei das wieder die Beschleunigung zu dem Zeitpunkt ist, der genau in der Mitte zwischen den Zeitpunkten liegt, für die die Geschwindigkeiten berechnet wurden.
Also wäre die Rechnung schonmal für sich linear mit der Zeit ändernde Beschleunigungen und Geschwindigkeiten exakt und der Rechenaufwand ist kaum gestiegen
Das Verlet verfahren lässt sich auch über eine Taylorentwicklung zur 3. Ordnung von der Mitte in beide Richtungen herleiten, der Fehler liegt damit erst in 4. Ordnung. Die antisymmetrische Anordnung des 1. und 3. Terms lässt sich diese wegkürzen, sodass der Rechenaufwand niedrig bleibt.
Da das Verlet-Verfahren ein symplektisches ist bleibt beim Verlet-Verfahren das Volumen des Phasenraums erhalten. Dies haben wir versucht zu demonstireren im Laufe des Projekts.

## Zeitschritte, Einheiten und Particle Class

Beim Verlet-Algorithmus werden für $n$ Iterationen mit dem Zeitschritt $h$ die Positionen $\vec{r}$ und Geschwindigkeiten $\vec{v}$ der Massenpunkte ("Particle") mit folgenden Zeitschritten ausgerechnet:
$$ \vec{r}(t_0 + h \cdot i) $$
$$ \vec{v}(t_0 - \frac{h}{2} + h \cdot i) $$
wobei
$$ i \in {1,...,n} $$
Die Werte für $\vec{r}(t)$ und $\vec{v}(t)$ eines Particle werden dabei in folgendermaßen berechnet:
$$ \vec{r}(t) = \vec{r}(t-h) + h \cdot \vec{v}(t-\frac{h}{2})$$
$$ \vec{v}(t) = \vec{v}(t-h) + h \cdot \vec{a}(t-\frac{h}{2})$$
Dabei ist $\vec{a}(t)$ die Beschleunigung des Particle, die aus $\vec{r}(t)$ folgendermaßen berechnet wird:
$$ \vec{a} = \frac{\vec{F}}{m} $$
$$ \vec{F} = G \cdot m \cdot \sum_{j=1}^p\frac{m_j}{|\vec{r_j}-r|^2}$$
$$ \Rightarrow \vec{a} = G \cdot \sum_{j=1}^p\frac{m_j}{|\vec{r_j}-r|^2}$$
Dabei sind $m_j$ die Massen und $\vec{r_j}$ die Ortsvektoren der $p$ anderen Körper im System.
Um das ständige Multiplizieren mit der Gravitationskonstante $G$ wärend der Simulation zu vermeiden, können wir stattdessen einfach zu Beginn alle Massen jeweis mit ihr multiplizieren und diese Größe $\lambda$ statt der Masse unter particle.mass speichern. Dann berechnet sich die Beschleunigung folgendermaßen:
$$ \vec{a} = \sum_{j=1}^p\frac{\lambda_j}{|\vec{r_j}-r|^2}$$
mit $$ \lambda_j = m_j \cdot G $$
Dabei sind $\lambda_j$ die mit der Gravitationskonstante multiplizierten massen particle.mass.
Die Massen werden im Code aber auch für die Berechnung der kinetischen und potentiellen Energie verwendet, also muss bei diesen Berechnungen die Masse durch $\frac{\lambda}{G}$ ersetzt werden:
$$ W_{pot}=G\frac{-m_1 \cdot m_2}{|\vec{r_1}-\vec{r_2}|} = \frac{1}{G} \cdot \frac{-\lambda_1 \cdot \lambda_2}{|\vec{r_1}-\vec{r_2}|}$$
$$ W_{kin}=\frac{1}{2} \cdot m \cdot v^2 = \frac{1}{G} \cdot \lambda \cdot v^2$$
Bei diesen Formeln sieht man dann, dass man die Gravitationskonstante einfach weglassen kann, und alle Energien werden mit dem gleichen Faktor skaliert, sodass alle Verhältnisse noch Stimmen.

In [ ]:
from Particle import Particle

myParticle1 = Particle("test1", 1, {"x": 0, "y": 0, "z": 0}, {"x": 2, "y": 0, "z": 0}, "#000000")
myParticle2 = Particle("test2", 1, {"x": 1, "y": 1, "z": 1}, {"x": 2, "y": 0, "z": 0}, "#000000")
print("initial:")
print(myParticle1)
print(myParticle2)
print("----------")
dt = 1.5
print("calculate movement for", dt, "seconds")
myParticle1.move(dt)
print("----------")
print("after:")
print(myParticle1)
print(myParticle2)


## Particle Factory

Im folgenden definieren wir einige Sets an Himmelskörpern -> "Particles" für die folgenden Simulationen und
eine Methode um die Geschwindigkeiten von Particles in einer Liste so zu "shiften", dass der Schwerpunkt des Systems konstant ist. 


In [ ]:
from ParticleFactory import ParticleFactory

testFactory = ParticleFactory("testName")
testFactory.importParticles([myParticle1, myParticle2])
print(testFactory)
testFactory.removeOffsetSpeed()
print("... removed offset speed ...")
print(testFactory)


## Simulate

To the simulate class you can add different simulation algorithms, that you pass a particle List and it will return a touple of coordinates (positions over time etc.) for plotting

In [ ]:
from Simulate import Simulate
testSimulate = Simulate(n = 2, h=0.1)
print(testSimulate.verlet(testFactory.particleList))

# Animate Class

Diese Klasse bekommt eine particleFactory übergeben, aus dieser werden die Particle, deren Farben und Größen gewonnen.<br>
Bekommt eine simulator Funktion wie etwa verlet() aus der Simulate class übergeben. Das zu ihr gehörige simulator Objekt, kennt die Anzahl und Größe der Zeitschritte n und h, so dass die simulator Funktion nur die partliceList übergeben bekommen muss.

Die Simulator Methode Verlet in der Simulat class gibt wie oben gesehen allerlei Werte zurück, die für das Plotten relevant sind.

In [ ]:
from Animate import Animate


## Execute

1. Erstelle eine ParticleFactory mit einer Particle List
2. Stelle sicher, dass diese keinen gesamt Impuls hat
3. Erstelle eine Simulate umgebung und definiere hierbei die Schrittgrößen und Anzahl an Schritten
4. Übergebe die ParticleFactory und Simulate umgebung an animate, so dass die Berechnung und danach das Plotten gestartet wird

#### einfache Elipse

In [ ]:
p = ParticleFactory("Elipse")
p.removeOffsetSpeed()
s = Simulate(h=0.1, n=10000)
a = Animate(p, s.verlet)

a.pathPlot()
a.energyPlot()
a.phaseSpace(True) #True to show 3d, False to show 2d

#### 2 Sonnen System

In [ ]:
p = ParticleFactory("Tatoo")
p.removeOffsetSpeed()
s = Simulate(h=10, n=1000)
a = Animate(p, s.verlet)

a.pathPlot()
a.energyPlot()
a.phaseSpace(True) #True to show 3d, False to show 2d
a.closePlot()

#### Moon System

In [ ]:
p = ParticleFactory("Moon System")
p.removeOffsetSpeed()
s = Simulate(h=0.15, n=3000)
a = Animate(p, s.verlet)

a.pathPlot()
a.energyPlot()
a.phaseSpace(True) #True to show 3d, False to show 2d
a.closePlot()

#### Solarsystem

In [36]:
p = ParticleFactory("Solar")
p.removeOffsetSpeed()
s = Simulate(h=5000, n=100000)
a = Animate(p, s.verlet)

a.pathPlot()
a.energyPlot()
a.phaseSpace(True) #True to show 3d, False to show 2d
a.closePlot()